In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Essential_Thrombocythemia/GSE65161'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Mediator kinase inhibition further activates super-enhancer-associated genes in AML"
!Series_summary	"Super-enhancers (SEs), which are composed of large clusters of enhancers densely loaded with the Mediator complex, transcription factors and chromatin regulators, drive high expression of genes implicated in cell identity and disease, such as lineage-controlling transcription factors and oncogenes. BRD4 and CDK7 are positive regulators of SE-mediated transcription. By contrast, negative regulators of SE-associated genes have not been well described. Here we show that the Mediator-associated kinases cyclin-dependent kinase 8 (CDK8) and CDK19 restrain increased activation of key SE-associated genes in acute myeloid leukaemia (AML) cells. We report that the natural product cortistatin A (CA) selectively inhibits Mediator kinases, has anti-leukaemic activity in vitro and in vivo, and disproportionately induces upregulation of SE-associated genes in CA

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = True  # Based on the context, let's assume it's gene expression data

# Data availability for essential thrombocythemia
trait_row = None  # No information indicating this variable is available in Sample Characteristics Dictionary

# Data availability for age
age_row = None  # No specific key indicating age data is available in Sample Characteristics Dictionary

# Data availability for gender
gender_row = None  # No specific key indicating gender data is available in Sample Characteristics Dictionary

# Data conversion functions
def convert_trait(value):
    return None  # Since trait_row is None, this function will not be used

def convert_age(value):
    try:
        return float(value.split(": ")[1])
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    try:
        val = value.split(": ")[1].lower()
        if val == 'male':
            return 1
        elif val == 'female':
            return 0
        else:
            return None
    except IndexError:
        return None

# Save cohort information
save_cohort_info('GSE65161', './preprocessed/Essential_Thrombocythemia/cohort_info.json', is_gene_available, trait_row is not None)

# Since trait_row is None, we skip the Clinical Feature Extraction substep
